In [1]:
!pip install -q peft accelerate bitsandbytes transformers datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import json

df = pd.read_csv("dataset_qa.csv")

examples = []
for _, row in df.iterrows():
    instruction = row["question"].strip()
    output = row["answer"].strip()
    if instruction and output:
        examples.append({"instruction": instruction, "output": output})

with open("career_data.jsonl", "w", encoding="utf-8") as f:
    for ex in examples:
        json.dump(ex, f, ensure_ascii=False)
        f.write("\n")


In [4]:
!pip install -U trl


In [15]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, TrainingArguments,
    DataCollatorForLanguageModeling, BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset


dataset = load_dataset("json", data_files="career_data.jsonl", split="train")


model_name = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = prepare_model_for_kbit_training(model)


peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)


training_args = TrainingArguments(
    output_dir="./phi2-career-model",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,
    save_strategy="epoch",
    report_to="none"
)

def formatting_func(example):
    prompt = f"### Question:\n{example['instruction']}\n\n### Answer:\n"
    answer = example["output"]
    return prompt + answer

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


trainer = SFTTrainer(
  model=model,
  train_dataset=dataset,
  args=training_args,
  data_collator=data_collator,
  formatting_func=formatting_func,
)

trainer.train()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Applying formatting function to train dataset:   0%|          | 0/1620 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1620 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1620 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1620 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1620 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recom

Step,Training Loss
10,1.796800
20,1.345900
30,1.179300
40,1.146700
50,1.095300
60,1.045600
70,0.985100
80,1.031800
90,1.000300
100,0.951300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=606, training_loss=0.8784051248342684, metrics={'train_runtime': 935.419, 'train_samples_per_second': 5.196, 'train_steps_per_second': 0.648, 'total_flos': 5107993296445440.0, 'train_loss': 0.8784051248342684})

In [16]:
!zip -r phi2-career-model.zip ./phi2-career-model


updating: phi2-career-model/ (stored 0%)
updating: phi2-career-model/checkpoint-606/ (stored 0%)
updating: phi2-career-model/checkpoint-606/scaler.pt (deflated 60%)
updating: phi2-career-model/checkpoint-606/rng_state.pth (deflated 25%)
updating: phi2-career-model/checkpoint-606/vocab.json (deflated 59%)
updating: phi2-career-model/checkpoint-606/optimizer.pt (deflated 8%)
updating: phi2-career-model/checkpoint-606/added_tokens.json (deflated 84%)
updating: phi2-career-model/checkpoint-606/adapter_model.safetensors (deflated 8%)
updating: phi2-career-model/checkpoint-606/tokenizer_config.json (deflated 94%)
updating: phi2-career-model/checkpoint-606/adapter_config.json (deflated 55%)
updating: phi2-career-model/checkpoint-606/scheduler.pt (deflated 56%)
updating: phi2-career-model/checkpoint-606/merges.txt (deflated 53%)
updating: phi2-career-model/checkpoint-606/tokenizer.json (deflated 82%)
updating: phi2-career-model/checkpoint-606/README.md (deflated 66%)
updating: phi2-career-mode

In [17]:
from google.colab import files
files.download("phi2-career-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
from transformers import pipeline

from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, "./phi2-career-model/checkpoint-606")
model.eval()

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)


question = "Describe the directions to work in the AI field?"

prompt = f"### Question:\n{question.strip()}\n\n### Answer:\n"

response = generator(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)[0]["generated_text"]


print("Your question:", question)
print("\nAnswer:")
print(response[len(prompt):].strip())


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

Your question: Describe the directions to work in the AI field?

Answer:
To enter the AI field, pursue a degree in computer science, machine learning, or related fields. Look for internships or research projects that involve AI and build your portfolio. Apply for jobs at tech companies specializing in AI development or data science.
